# Introduction to Nengo SPA

This tutorial introduces the usage of Nengo SPA. It expects some basic familarity with [Nengo](https://pythonhosted.org/nengo/index.html). If you have used the legacy SPA implementation shipped with core Nengo, you might want to read this alternate introduction (TODO link).

In [ ]:
%matplotlib inline
%load_ext nengo.ipynb

We recommend to `import nengo_spa as spa`. (Note that this uses an underscore in the module name and is different from `nengo.spa` which refers to the legacy SPA module shipped with core Nengo.)

In [ ]:
import matplotlib.pyplot as plt
import nengo
import nengo_spa as spa
import numpy as np

We will have to specify the dimensionality of the Semantic Pointers. To make it easy to change in all places, we define the variable *d* here and set it to 32. A dimensionality of 32 is on the lower end (in most actual models you will want to use at least 64 dimensions and we have been using up to 512 dimensions), but it makes the examples in this introduction run faster.

In [ ]:
d = 32

## Hello world

Let us start with a very simple model to demonstrate the basic usage of Nengo SPA:

In [ ]:
with spa.Network() as model:
    model.stimulus = spa.Encode('Hello', vocab=d)
    model.state = spa.State(vocab=d)
    nengo.Connection(model.stimulus.output, model.state.input)
    p = nengo.Probe(model.state.output, synapse=0.01)

The first thing to notice is that instead of `nengo.Network`, we use `spa.Network` here. This allows use to more easily use Nengo's config system for Nengo SPA (something we will look at in more detail later). Then we instantiate two networks `spa.Encode` and `spa.State`. These networks are aware of Semantic Pointer inputs and/or outputs. Such networks we also call (SPA) modules. The `Encode` module is similar to a `nengo.Node`. Here it is given a the constant Semantic Pointer *Hello* and it will output this pointer during the whole simulation. The `State` module is a network of Nengo ensembles that is optimized for representing (unit-length) Semantic Pointers. Both of these modules have a *vocab* argument which is short for *vocabulary*. In the context of Nengo SPA a vocabulary is a set of Semantic Pointers with a certain dimensionality. Here we just use the default vocabulary with dimensionality *d*. The required Semantic Pointers (in this example *Hello*) will be automatically added to that vocabulary.

Modules can be used like normal Nengo networks. Thus, we can create a connection from the output of *stimulus* to the input of *state* and then probe the output of *state*.

In [ ]:
with nengo.Simulator(model) as sim:
    sim.run(0.5)

Now, let us plot the probed data:

In [ ]:
plt.plot(sim.trange(), sim.data[p])
plt.xlabel("Time [s]");

This plot displays the raw vector components of the represented Semantic Pointer and is not extremely helpful. A useful function to get a more informative plot is `spa.similarity`. It takes the probe data and a vocabulary as arguments, and returns the similarity of the data to each Semantic Pointer in the vocabulary. We can access the vocabulary with the *vocab* attribute of the `State` module.

In [ ]:
plt.plot(sim.trange(), spa.similarity(sim.data[p], model.state.vocab))
plt.xlabel("Time [s]")
plt.ylabel("Similarity")
plt.legend(model.state.vocab, loc='best');